In [1]:
import csv
import random

import nltk
import pandas as pd
import scipy.stats

import Ent

#first run python manage.py shell_plus --notebook and   'django_extensions' is in INSTALLED_APPS.

In [5]:
import pickle

l = pickle.load( open( "deep_pickle.p", "rb" ) )

In [161]:
"""
grab entry data from DEEP

----

context: list
population profile: list
communication: list
humanitarian access: list
sector, pillar + subpillar: dict w/ lists... {sector: [subsect]...}
reliability: int
severity: int
demo groups: list
specific needs: list
affected groups: list
geo locations: list (blank for now)
subsector: dict w/ lists... {sector: [subsect]...}
information date: date
excerpt: string
has_image: T/F
lead: int
"""

from entries.models import Entry
from entries.models import Sector

def extract():
    """iterate through each entry and extract information"""
    deep_ents = []
    
    for ent_it in Entry.objects.all():

        #pillar/subpillar/subsector. if just top row, second two are None

        #context, population, communication, humanitarian access, pillar/subpillar
        for subent in ent_it.entryinformation_set.all():
            cent = Ent()

            for info_att in subent.informationattribute_set.all():
                #pillar/subpillar
                if info_att.sector:
                    cent.add_twodim(info_att)

                #context, population, communication, humanitarian access
                else:
                    cent.add_onedim(info_att)

            deep_ents.append(cent)

            #reliability
            cent.reliability = subent.reliability.name

            #severity
            cent.severity = subent.reliability.name

            #demograhpic groups
            if len(subent.vulnerable_groups.all()) > 0:
                cent.demo_groups = [n.name for n in subent.vulnerable_groups.all()]

            #specific
            if len(subent.specific_needs_groups.all()) > 0:
                cent.specific_needs = [n.name for n in subent.specific_needs_groups.all()]

            #affected
            if len(s.affected_groups.all()) > 0:
                cent.affected_groups = [g.name for g in subent.affected_groups.all()]

            #geo - none for now

            #information date
                cent.information_date = subent.date

            #excerpt
                cent.excerpt = subent.excerpt

            #has_image
                cent.has_image = subent.image == ''

            #lead id
                cent.lead = ent_it.lead_id

            #lead content (passed for now to save space)
                #cent.lead = ent_it.lead.simplifiedlead.text

            #event
            cent.event = ent_it.lead.event.name
        
    return deep_ents

In [162]:
#pickle
import pickle 

def pickle_rick():
    #open with pickle.load( open( "save.p", "rb" ) )
    prepare_to_get_ricked = extract()
    pickle.dump(prepare_to_get_ricked, open('./deep_pickle.p', 'wb'))
    
pickle_rick()

In [6]:
#data input
"""
context: list
population profile: list
communication: list
humanitarian access: list
sector, pillar + subpillar: dict w/ lists... {sector: [subsect]...}
reliability: int
severity: int
demo groups: list
specific needs: list
affected groups: list
geo locations: list
subsector: dict w/ lists... {sector: [subsect]...}
information date: date
excerpt: string
has_image: 
"""
from entries.models import Entry
from entries.models import Sector

#get lookups for sectors
SECT_LOOK = {i.id : i.name for i in models.Sector.objects.all()}


def extract_sects(raw):
	sect_dict = {}
	for v in raw:
		#there can be multiple sectors in an entry so we have to cycle through and get them
		#since we can have sub-pillars, we take the unique sector types

		for uniq in list(set(SECT_LOOK[k['sector_id']] for k in v.informationattribute_set.values())):
			if v.pk not in sect_dict:
				sect_dict[v.pk] = {v.excerpt : [uniq]}
			else:
				sect_dict[v.pk][v.excerpt].append(uniq)

	return sect_dict

def create_csv(data, file_name):
    with open(file_name, 'w') as f:
        csv_writer = csv.writer(f)
        for k,v in data.items():
            csv_writer.writerow([k, v])

l = [i.entryinformation_set.all() for i in Entry.objects.all()]
div = [sub for list in l for sub in list]

extract_sects(div)

NameError: name 'models' is not defined

In [ ]:
#data prep

#read in csv, make dict with cats
#structure: 0 country | 1 date | 2 sect | 3 subsect | 4 text

rd = csv.reader(open('/Users/ewanog/code/repos/deep/out'))

cats = {}

for row in rd:
    sect = row[0]
    txt = row[1:]
    if sect not in cats:
        cats[sect] = txt
    else:
        cats[sect].extend(txt)

#counts
c=0
for k,v in cats.items():
    print(k + ' ' + str(len(v)))
    c+=len(v)
print('total count: ' + str(c))
    
print('all: ' + str(len(cats.values())))
    
cats    
    
#TODO: clean bad chars

#remove stopwords. STOP!!!!!!!!!ADSAAA!!W

#lower - then remove all the other calls

In [ ]:
#show duplicates
from pprint import pprint

c=0
for k,v in cats.items():
    print(k + ' ' + str(len((set(v)))))
    c+=len((set(v)))
           
print(c)

In [ ]:
#create random sampling groups
#TODO: should we do uniform random sampling per group or a whole scale random draw?

#return dicts with {sect: setted list of entries}
def gen_tt(corpus):
    test = {}
    train = {}

    #add corpus to test and train
    for k in corpus.keys():
        test[k] = None
        train[k] = None

    for k,v in corpus.items():
        random.shuffle(v)

        setv = list(set(v))
        len_test = int(len(setv)*.3)

        test[k] = setv[:len_test]
        train[k] = setv[len_test:]

    return train, test

train, test = gen_tt(cats)
#     #check lengths
#     for k in train.keys():
#         print(k + ' train : ' + str(len(train[k])))
#         print(k + ' test : ' + str(len(test[k])))

In [ ]:
#test sampling (if we do 70:30)

d = {
    'cat1': [random.randint(0,100) for i in range(100)],
    'cat2': [random.randint(0,100) for i in range(150)]
}

test_test = {}
test_train = {}

#add cats to test and train
for k in d.keys():
    test_test[k] = None
    test_train[k] = None

for k,v in d.items():
    len_test = int(len(v)*.3)
    print(len(v)*.3)
    
    test_test[k] = v[:len_test]
    test_train[k] = v[len_test:]
    
print(len(test_test))
print(len(test_train))

In [ ]:
train_merge

In [ ]:
#start building our model
#train and test are just subsets of cats (for now)

#make a feature with word presence
#TODO: what's python func?
all_words = []
for k,v in cats.items():
    for e in list(set(v)):
        for w in e.split(' '):
            all_words.append(w.lower())

freq_words = nltk.FreqDist(all_words)

def document_features(document):
    if type(document) == list:
        #TODO: func
        grp = ''
        for l in document:
            for w in l:
                grp += w
        document = grp
                
    uniq_doc = set(document.split(' '))
    features = {}
    for word in list(freq_words.keys())[:2000]:
        features['contains(%s)' % word] = (word in uniq_doc)
    return features

#convert dicts into list of tuples we need
#TODO: func
train_merge = []
test_merge = []

for k,v in train.items():
    for i in v:
        train_merge.append((k,i))

for k,v in test.items():
    for i in v:
        test_merge.append((k,i))

        
print(len(test_merge))

wp_train = [(document_features(v), k) for k,v in train_merge]
wp_test = [(document_features(v), k) for k,v in test_merge]

nb_class = nltk.NaiveBayesClassifier.train(wp_train)

# wp_test[0]

In [ ]:
#results

#from just raw, unrefined features and 200 words: 0.398576512455516

nb_class.show_most_informative_features(100)

In [ ]:
# ['contains(hit)',
#  'contains(down)',
#  'contains(being)',
#  'contains(member.)',
#  'contains(17)',
#  'contains(other)',
#  'contains(air)',
#  'contains(redirect,)',
#  'contains(released)',
#  'contains(independence)',
#  'contains(until)',
#  'contains(the)']

# [k for k,v in wp_train[100][0].items() if v is True]

wp_train[100][1]

In [ ]:
for k,v in train.items():
    print(k)
    print(v)
    break

In [ ]:
wpfeatures[0][0]